<a href="https://colab.research.google.com/github/Glitch0000/Experimenting_with_custom_loss_functions/blob/main/Experimenting_with_custom_loss_functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creating a Custom Loss Function
In this notbook we will create custom loss functions. We will build a simple linear regression neural network in order to test our custom functions. The data are two arrays : $xs$ (inputs) and $ys$ (labels), where the relationship between each corresponding element is $y=3x-1$.:

$xs = [-1.0,  0.0, 1.0, 2.0, 3.0, 4.0]$

$ys = [-4.0, -1.0, 2.0, 5.0, 8.0, 11.0]$



In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import backend as k

In [2]:
# inputs
xs = np.array([-1.0,  0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)

# labels. relationship with the inputs above is y=3x-1.
ys = np.array([-4.0, -1.0, 2.0, 5.0, 8.0, 11.0], dtype=float)

### Defining the custom loss function 
First, we create a simpe loss function that returns the root mean square error (RMSE).It is called `my_rmse()` , returns the RMSE between the target (`y_true`) and prediction (`y_pred`). 

You will return $\sqrt{error}$, where $error$ = $mean((y_{true} - y_{pred})^2)$
- error: the difference between the true label and predicted label.
- sqr_error: the square of the error.
- mean_sqr_error: the mean of the square of the error
- sqrt_mean_sqr_error: the square root of hte mean of the square of the error (the root mean squared error).


In [3]:
def my_rmse(y_true, y_pred):
    error = y_true-y_pred
    sqr_error = k.square(error) 
    mean_sqr_error = k.mean(sqr_error)
    sqrt_mean_sqr_error = k.sqrt(mean_sqr_error)
    return sqrt_mean_sqr_error

### Defining a model using the custom loss function 
Here we define a simple model and pass the function you just coded as the loss.

In [10]:
model = tf.keras.Sequential([keras.layers.Dense(units=1, input_shape=[1])])

model.compile(optimizer='sgd', loss=my_rmse)
              
# train the model 
model.fit(xs, ys, epochs=500,verbose=0)
              
# test with a sample input
print(model.predict([10.0]))

[[28.908419]]


###Huber loss function

In [6]:
def my_huber_loss_with_threshold(threshold):
  
    # function that accepts the ground truth and predictions
    def my_huber_loss(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) <= threshold
        small_error_loss = tf.square(error) / 2
        big_error_loss = threshold * (tf.abs(error) - (0.5 * threshold))
        
        return tf.where(is_small_error, small_error_loss, big_error_loss) 

    # return the inner function tuned by the hyperparameter
    return my_huber_loss

##Testing the loss function

In [7]:
model = tf.keras.Sequential([keras.layers.Dense(units=1, input_shape=[1])])
model.compile(optimizer='sgd', loss=my_huber_loss_with_threshold(threshold=10))
model.fit(xs, ys, epochs=500,verbose=0)
print(model.predict([10.0]))

[[28.670618]]


### Building Huber loss function as a class

In [8]:
from tensorflow.keras.losses import Loss

class MyHuberLoss(Loss):
  
    # initialize instance attributes
    def __init__(self, threshold=1):
        super().__init__()
        self.threshold = threshold

    # compute loss
    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) <= self.threshold
        small_error_loss = tf.square(error) / 2
        big_error_loss = self.threshold * (tf.abs(error) - (0.5 * self.threshold))
        return tf.where(is_small_error, small_error_loss, big_error_loss)

###Testing the loss function
We specify the loss by instantiating an object from your custom loss class.

In [9]:
model = tf.keras.Sequential([keras.layers.Dense(units=1, input_shape=[1])])
model.compile(optimizer='sgd', loss=MyHuberLoss(threshold=10))
model.fit(xs, ys, epochs=500,verbose=0)
print(model.predict([10.0]))

[[28.680162]]


# We notice that RMSE loss function gives much better resuts in this problem than Huber loss function.